# CSE 258 - HW 3
Jin Dai / A92408103

## Task (Cook/Make prediction)

In [2]:
import gzip
import random
from collections import defaultdict
import csv

In [3]:
def read_gz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def read_csv(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    print(header)
    for l in c:
        yield l

In [4]:
dataset = list(read_csv("trainInteractions.csv.gz"))
dataset[:2]

['user_id', 'recipe_id', 'date', 'rating']


[['88348277', '03969194', '2004-12-23', '5'],
 ['86699739', '27096427', '2002-01-12', '4']]

In [5]:
# shuffle
random.shuffle(dataset)

In [6]:
train_size = 400000
data_train = dataset[:train_size]
data_valid = dataset[train_size:]
print('data_train size = %d\tdata_valid size = %d' % (len(data_train), len(data_valid)))
X_train = [d[:2] for d in data_train]
X_valid = [d[:2] for d in data_valid]

data_train size = 400000	data_valid size = 100000


### 1.

In [7]:
# compute user-recipes dict and all recipes set
user_recipes = defaultdict(set)
all_recipes = set()
for d in X_train:
    usr = d[0]
    r = d[1]
    all_recipes.add(r)
    user_recipes[usr].add(r)

In [8]:
# get a negative sample per each entry in the validation set
def random_sample(from_list, exclusions):
    s = random.choice(from_list)
    while s in exclusions:
        s = random.choice(from_list)
    return s

all_recipes_list = list(all_recipes)
not_made_samples = []
for d in X_valid:
    usr = d[0]
    r_cooked = d[1]
    r_uncooked = random_sample(all_recipes_list, user_recipes[usr].union({r_cooked}))
    not_made_samples.append([usr, r_uncooked])

In [9]:
# the baseline model
recipe_count = defaultdict(int)
total_cooked = 0

for d in X_train:
    r = d[1]
    recipe_count[r] += 1
    total_cooked += 1

most_popular = [(recipe_count[x], x) for x in recipe_count]
most_popular.sort()
most_popular.reverse()

def fit(most_popular, total_cooked, threshold=0.5):
    popular_set = set()
    count = 0
    for ic, i in most_popular:
      count += ic
      popular_set.add(i)
      if count > total_cooked * threshold: break
    return popular_set

return1 = fit(most_popular, total_cooked)

In [10]:
def predict_accuracy_1(made, not_made, popular_set):
    # count the true positives
    TP_count = 0
    for d in made:
        r = d[1]
        if r in popular_set:
            TP_count += 1
    # count the true negatives
    TN_count = 0
    for d in not_made:
        r = d[1]
        if r not in popular_set:
            TN_count += 1
    print('true positive = %d' % TP_count)
    print('true negative = %d' % TN_count)
    return (TP_count + TN_count) / (1.0 * (len(made) + len(not_made)))

In [11]:
accuracy = predict_accuracy_1(X_valid, not_made_samples, return1)
print('given threshold=0.5, overall accuracy on the validation set + negative sample set is %.5f' % accuracy)

true positive = 44782
true negative = 87971
given threshold=0.5, overall accuracy on the validation set + negative sample set is 0.66377


### 2.

First we perform random sampling on all user-recipe pairs to get the "non-made" examples.

In [12]:
recipe_users = defaultdict(set)
all_users = set()
for d in X_train:
    usr = d[0]
    r = d[1]
    all_users.add(usr)
    recipe_users[r].add(usr)

In [13]:
def random_sample_2(all_users, all_recipes, exclusions):
    usr = random.choice(all_users)
    r = random.choice(all_recipes)
    s = (usr, r)
    while s in exclusions:
        usr = random.choice(all_users)
        r = random.choice(all_recipes)
        s = (usr, r)
    return s

all_cooked_user_recipe_pair = set([(d[0], d[1]) for d in X_train + X_valid])
all_users_list = list(all_users)

not_made_samples = set()
while len(not_made_samples) < len(X_valid):
    not_made_samples.add(random_sample_2(all_users_list, all_recipes_list, all_cooked_user_recipe_pair))
not_made_samples = list(not_made_samples)

For this question, we can try calculating accuracies using a few different thresholds like 10th, 20th,...,80th, 90th percentiles.
We also want to compare against the accuracy using all the training data.

In [14]:
max_accuracy = 0
best_threshold = 0.5
for t in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    return_n = fit(most_popular, total_cooked, t)
    accuracy = predict_accuracy_1(X_valid, not_made_samples, return_n)
    print('given threshold=%.2f, overall accuracy on the validation set + negative sample set is %.5f' % (t, accuracy))
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_threshold = t

true positive = 9829
true negative = 99673
given threshold=0.10, overall accuracy on the validation set + negative sample set is 0.54751
true positive = 19401
true negative = 98692
given threshold=0.20, overall accuracy on the validation set + negative sample set is 0.59047
true positive = 28427
true negative = 96594
given threshold=0.30, overall accuracy on the validation set + negative sample set is 0.62511
true positive = 36744
true negative = 93051
given threshold=0.40, overall accuracy on the validation set + negative sample set is 0.64897
true positive = 44782
true negative = 87874
given threshold=0.50, overall accuracy on the validation set + negative sample set is 0.66328
true positive = 52622
true negative = 80584
given threshold=0.60, overall accuracy on the validation set + negative sample set is 0.66603
true positive = 60031
true negative = 70312
given threshold=0.70, overall accuracy on the validation set + negative sample set is 0.65172
true positive = 67266
true negative

In [15]:
print("When we set the threshold to the %dth percentile, the max accuracy is reached at %.5f." % (best_threshold * 100, max_accuracy))

When we set the threshold to the 60th percentile, the max accuracy is reached at 0.66603.


### 3.

In [16]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return (1.0 * numer) / denom

In [17]:
max_sim_dict = defaultdict(float)

def get_max_sim(usr, r):
    if (usr, r) in max_sim_dict:
        return max_sim_dict[(usr, r)]
    max_sim = 0.0
    for r2 in user_recipes[usr]:
        if r2 == r: continue
        max_sim = max(max_sim, Jaccard(recipe_users[r], recipe_users[r2]))
    max_sim_dict[(usr, r)] = max_sim
    return max_sim

def predict_accuracy_3(made, not_made, sim_threshold):
    TP_count = 0
    for d in made:
        if get_max_sim(d[0], d[1]) > sim_threshold:
            TP_count += 1
    TN_count = 0
    for d in not_made:
        if get_max_sim(d[0], d[1]) <= sim_threshold:
            TN_count += 1
    print('true positive = %d' % TP_count)
    print('true negative = %d' % TN_count)
    print(len(max_sim_dict))
    return (TP_count + TN_count) / (1.0 * (len(made) + len(not_made)))

In [18]:
accuracy = predict_accuracy_3(X_valid, not_made_samples, 0.5)
print('given threshold=0.5, overall accuracy on the validation set + negative sample set is %.5f' % accuracy)

true positive = 31
true negative = 100000
200000
given threshold=0.5, overall accuracy on the validation set + negative sample set is 0.50016


In [19]:
import statistics
# get the median of all Jaccard similarities in max_sim_dict
max_sims = [v for _, v in max_sim_dict.items()]
max_sims.sort()
statistics.median(max_sims)

0.0

In [20]:
len(max_sims)

200000

In [21]:
for t in [0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.1, 0.15, 0.2]:
    accuracy = predict_accuracy_3(X_valid, not_made_samples, t)
    print('given threshold=%.3f, overall accuracy on the validation set + negative sample set is %.5f' % (t, accuracy))
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_threshold = t

true positive = 64720
true negative = 78550
200000
given threshold=0.000, overall accuracy on the validation set + negative sample set is 0.71635
true positive = 64720
true negative = 78550
200000
given threshold=0.001, overall accuracy on the validation set + negative sample set is 0.71635
true positive = 63473
true negative = 80946
200000
given threshold=0.005, overall accuracy on the validation set + negative sample set is 0.72210
true positive = 61331
true negative = 83657
200000
given threshold=0.010, overall accuracy on the validation set + negative sample set is 0.72494
true positive = 55681
true negative = 87426
200000
given threshold=0.020, overall accuracy on the validation set + negative sample set is 0.71554
true positive = 48525
true negative = 89664
200000
given threshold=0.030, overall accuracy on the validation set + negative sample set is 0.69095
true positive = 40768
true negative = 91474
200000
given threshold=0.040, overall accuracy on the validation set + negative 

In [22]:
print("When we set the similarity threshold to the %dth percentile, the max accuracy is reached at %.5f." % (best_threshold * 100, max_accuracy))

When we set the similarity threshold to the 1th percentile, the max accuracy is reached at 0.72494.


### 4

In [23]:
# define the predict function
def single_predict(user, recipe, popular_set, sim_threshold):
    if recipe in popular_set or get_max_sim(user, recipe) > sim_threshold:
        return 1
    return 0

def predict(X, popular_set, sim_threshold):
    return [single_predict(d[0], d[1], popular_set, sim_threshold) for d in X]

In [24]:
def predict_accuracy_4(made, not_made, popular_set, sim_threshold=0.01):
    pred_made = predict(made, popular_set, sim_threshold)
    TP_count = pred_made.count(1)
    pred_not_made = predict(not_made, popular_set, sim_threshold)
    TN_count = pred_not_made.count(0)
    print('true positive = %d' % TP_count)
    print('true negative = %d' % TN_count)
    return (TP_count + TN_count) / (1.0 * (len(made) + len(not_made)))

In [25]:
max_accuracy = 0
best_t1 = 0.6
best_t2 = 0.01
for t1 in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    pop_set_perc_t1 = fit(most_popular, total_cooked, t1)
    for t2 in [0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75]:
        accuracy = predict_accuracy_4(X_valid, not_made_samples, pop_set_perc_t1, t2)
        print('Using popularity threshold=%.3f and similarity threshold=%.3f, we can achieve overall accuracy=%.5f on the validation set + negative sample set.' % (t1, t2, accuracy))
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_t1 = t1
            best_t2 = t2

true positive = 61846
true negative = 83569
Using popularity threshold=0.100 and similarity threshold=0.010, we can achieve overall accuracy=0.72708 on the validation set + negative sample set.
true positive = 55464
true negative = 88484
Using popularity threshold=0.100 and similarity threshold=0.025, we can achieve overall accuracy=0.71974 on the validation set + negative sample set.
true positive = 44551
true negative = 92313
Using popularity threshold=0.100 and similarity threshold=0.050, we can achieve overall accuracy=0.68432 on the validation set + negative sample set.
true positive = 38178
true negative = 94166
Using popularity threshold=0.100 and similarity threshold=0.075, we can achieve overall accuracy=0.66172 on the validation set + negative sample set.
true positive = 32689
true negative = 95805
Using popularity threshold=0.100 and similarity threshold=0.100, we can achieve overall accuracy=0.64247 on the validation set + negative sample set.
true positive = 15524
true neg

In [26]:
print("When we set the popularity threshold to the %.2fth percentile and similarity threshold to the %.2fth percentile,\nthe max accuracy is reached at %.5f." % (best_t1 * 100, best_t2 * 100, max_accuracy))

When we set the popularity threshold to the 40.00th percentile and similarity threshold to the 5.00th percentile,
the max accuracy is reached at 0.73662.


In [27]:
max_accuracy = 0
best_t1 = 0.6
best_t2 = 0.01
for t1 in [0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4]:
    pop_set_perc_t1 = fit(most_popular, total_cooked, t1)
    for t2 in [0.03, 0.035, 0.036, 0.037, 0.038, 0.039, 0.04, 0.041, 0.042, 0.043, 0.044, 0.045]:
        accuracy = predict_accuracy_4(X_valid, not_made_samples, pop_set_perc_t1, t2)
        print('Using popularity threshold=%.3f and similarity threshold=%.3f, we can achieve overall accuracy=%.5f on the validation set + negative sample set.' % (t1, t2, accuracy))
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            best_t1 = t1
            best_t2 = t2

true positive = 60190
true negative = 87149
Using popularity threshold=0.300 and similarity threshold=0.030, we can achieve overall accuracy=0.73669 on the validation set + negative sample set.
true positive = 59419
true negative = 87894
Using popularity threshold=0.300 and similarity threshold=0.035, we can achieve overall accuracy=0.73657 on the validation set + negative sample set.
true positive = 59243
true negative = 88033
Using popularity threshold=0.300 and similarity threshold=0.036, we can achieve overall accuracy=0.73638 on the validation set + negative sample set.
true positive = 59237
true negative = 88034
Using popularity threshold=0.300 and similarity threshold=0.037, we can achieve overall accuracy=0.73635 on the validation set + negative sample set.
true positive = 59065
true negative = 88210
Using popularity threshold=0.300 and similarity threshold=0.038, we can achieve overall accuracy=0.73638 on the validation set + negative sample set.
true positive = 58882
true neg

In [28]:
print("When we set the popularity threshold to the %.2fth percentile and similarity threshold to the %.2fth percentile,\nthe max accuracy is reached at %.5f." % (best_t1 * 100, best_t2 * 100, max_accuracy))

When we set the popularity threshold to the 34.00th percentile and similarity threshold to the 3.50th percentile,
the max accuracy is reached at 0.73742.


### 5

In [29]:
recipe_count = defaultdict(int)
total_cooked = 0

for d in dataset:
    r = d[1]
    recipe_count[r] += 1
    total_cooked += 1

most_popular = [(recipe_count[x], x) for x in recipe_count]
most_popular.sort()
most_popular.reverse()

In [30]:
best_pop_set = fit(most_popular, total_cooked, best_t1)
predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if single_predict(u, i, best_pop_set, best_t2):
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + ",0\n")

predictions.close()

Kaggle Username: Jin Dai